# Title 

Continuous Assessment 1 06/10/2023 - BD & ADA Module // MSc in Data Analytics Y1 S2 - <b>Student ID 2023063

## Introduction
write intro

--

Data source: https://www.kaggle.com/datasets/mkechinov/ecommerce-events-history-in-cosmetics-shop/data

## Practical Big Data (PySparkSQL) - Prep data

In [1]:
#Import PySparkSQL
import pyspark
from pyspark.sql import SparkSession        

# Create SparkSession
spark = (SparkSession
  .builder
  .appName("SparkSQL")
  .getOrCreate())

# Path to dataset
csv_file = "file:////home/hduser/Downloads/CA1/2020-Jan.csv"

In [2]:
# Read and create a temporary view
df = (spark.read.format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load(csv_file))
df.createOrReplaceTempView("rawdata")

In [3]:
# Visualise inferred schema
data = spark.sql("SELECT * FROM rawdata")
data.printSchema()

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [4]:
# Display 5 first rows
data.show(5)

+--------------------+----------+----------+-------------------+-------------+--------+-----+---------+--------------------+
|          event_time|event_type|product_id|        category_id|category_code|   brand|price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+-------------+--------+-----+---------+--------------------+
|2020-01-01 00:00:...|      view|   5809910|1602943681873052386|         null| grattol| 5.24|595414620|4adb70bb-edbd-498...|
|2020-01-01 00:00:...|      view|   5812943|1487580012121948301|         null|kinetics| 3.97|595414640|c8c5205d-be43-4f1...|
|2020-01-01 00:00:...|      view|   5798924|1783999068867920626|         null|  zinger| 3.97|595412617|46a5010f-bd69-4fb...|
|2020-01-01 00:00:...|      view|   5793052|1487580005754995573|         null|    null| 4.92|420652863|546f6af3-a517-475...|
|2020-01-01 00:00:...|      view|   5899926|2115334439910245200|         null|    null| 3.92|484071203|cff70ddf-529e-4b0...|


In [5]:
# Number of users
action = spark.sql("SELECT count(distinct user_id) FROM rawdata")
action.show()

+-----------------------+
|count(DISTINCT user_id)|
+-----------------------+
|                 410073|
+-----------------------+



In [6]:
# Number of events
action = spark.sql("SELECT count(distinct event_type) FROM rawdata")
action.show()

+--------------------------+
|count(DISTINCT event_type)|
+--------------------------+
|                         4|
+--------------------------+



In [7]:
# Number of products
action = spark.sql("SELECT count(distinct product_id) FROM rawdata")
action.show()

+--------------------------+
|count(DISTINCT product_id)|
+--------------------------+
|                     45484|
+--------------------------+



In [8]:
# Top seller product
countProduct = spark.sql("SELECT product_id, count(*) FROM rawdata GROUP BY product_id ORDER BY count(*) desc")
countProduct.show(5)

+----------+--------+
|product_id|count(1)|
+----------+--------+
|   5809910|   36967|
|   5809912|   14310|
|   5917178|   12634|
|   5700037|   11429|
|   5686925|   11098|
+----------+--------+
only showing top 5 rows



In [9]:
# New dataframe with topseller product
topseller = spark.sql("SELECT DISTINCT * FROM rawdata WHERE product_id LIKE 5809910")
topseller.show(5)

+--------------------+----------------+----------+-------------------+-------------+-------+-----+---------+--------------------+
|          event_time|      event_type|product_id|        category_id|category_code|  brand|price|  user_id|        user_session|
+--------------------+----------------+----------+-------------------+-------------+-------+-----+---------+--------------------+
|2020-01-01 00:00:...|            view|   5809910|1602943681873052386|         null|grattol| 5.24|595414620|4adb70bb-edbd-498...|
|2020-01-01 14:21:...|            view|   5809910|1602943681873052386|         null|grattol| 5.24|592285428|33ecf984-0438-459...|
|2020-01-01 16:56:...|            view|   5809910|1602943681873052386|         null|grattol| 5.24|595651372|121148f9-4e42-48b...|
|2020-01-01 18:40:...|remove_from_cart|   5809910|1602943681873052386|         null|grattol| 5.24|583351843|85fa8bff-02e8-4a5...|
|2020-01-02 06:09:...|            view|   5809910|1602943681873052386|         null|gratto

In [10]:
'''# Add new column with condition to topseller df
# If event_time is in the first couple of weeks of the month, column FN1 (fortnight1) is equals to 1 (True)
# If in the last 2 weeks of the month column FN1 is equals 0 (False)

from pyspark.sql.functions import when
from pyspark.sql.functions import lit

topseller = topseller.withColumn("FN1", \
   when((topseller.event_time < '2020-01-16'), lit(1)) \
     .when((topseller.event_time >= '2020-01-15'), lit(0)) \
                                )'''

'# Add new column with condition to topseller df\n# If event_time is in the first couple of weeks of the month, column FN1 (fortnight1) is equals to 1 (True)\n# If in the last 2 weeks of the month column FN1 is equals 0 (False)\n\nfrom pyspark.sql.functions import when\nfrom pyspark.sql.functions import lit\n\ntopseller = topseller.withColumn("FN1",    when((topseller.event_time < \'2020-01-16\'), lit(1))      .when((topseller.event_time >= \'2020-01-15\'), lit(0))                                 )'

In [11]:
'''# Show dataframe
topseller.show(5)'''

'# Show dataframe\ntopseller.show(5)'

In [12]:
# Drop columns
topseller = topseller.drop(*["product_id","category_id","category_code","brand","price"])

In [13]:
# Show updated topseller df
topseller.show(10,False)

+-----------------------+----------------+---------+------------------------------------+
|event_time             |event_type      |user_id  |user_session                        |
+-----------------------+----------------+---------+------------------------------------+
|2020-01-01 00:00:00 UTC|view            |595414620|4adb70bb-edbd-4981-b60f-a05bfd32683a|
|2020-01-01 14:21:02 UTC|view            |592285428|33ecf984-0438-459c-aea6-241319336d75|
|2020-01-01 16:56:51 UTC|view            |595651372|121148f9-4e42-48ba-94eb-7aca8935940b|
|2020-01-01 18:40:56 UTC|remove_from_cart|583351843|85fa8bff-02e8-4a5f-80b8-a129ce5555e3|
|2020-01-02 06:09:01 UTC|view            |529378930|b99ea2eb-1c67-4f39-ab20-792ebfb87f93|
|2020-01-02 08:21:31 UTC|view            |595842440|b257db78-93e8-46d4-9893-56194751a4d9|
|2020-01-03 06:16:01 UTC|view            |595798798|8b2d9742-eb2f-4bbd-8387-ace81cfdef2a|
|2020-01-03 11:17:22 UTC|cart            |596055322|74521846-45da-4a1e-a61e-19fbe7f6fb82|
|2020-01-0

In [14]:
# Customer behaviour before purchasing example
# Random customer collected from filtering event_type: 'purchase' 
from pyspark.sql.functions import asc
topseller.select("event_time","event_type","user_session").filter("user_id = 431397250").orderBy(asc("event_time")).show(truncate=False)

+-----------------------+----------+------------------------------------+
|event_time             |event_type|user_session                        |
+-----------------------+----------+------------------------------------+
|2020-01-07 14:33:37 UTC|view      |d6151353-322f-4a7a-b3a4-0c4c0e52e2f5|
|2020-01-07 14:39:42 UTC|cart      |d6151353-322f-4a7a-b3a4-0c4c0e52e2f5|
|2020-01-07 18:05:19 UTC|purchase  |697d43d2-07f9-4c59-aa7c-8dca123cebc0|
+-----------------------+----------+------------------------------------+



In [15]:
# Customer behaviour before purchasing example
# Random customer collected from filtering event_type: 'purchase' 
from pyspark.sql.functions import asc
topseller.select("event_time","event_type","user_session").filter("user_id = 46383252").orderBy(asc("event_time")).show(30,truncate=False)

+-----------------------+----------+------------------------------------+
|event_time             |event_type|user_session                        |
+-----------------------+----------+------------------------------------+
|2020-01-28 06:12:26 UTC|cart      |5549dfe6-109e-497f-a9f2-01c0aecdd73f|
|2020-01-28 06:14:08 UTC|view      |5549dfe6-109e-497f-a9f2-01c0aecdd73f|
|2020-01-28 06:17:17 UTC|view      |5549dfe6-109e-497f-a9f2-01c0aecdd73f|
|2020-01-28 06:53:25 UTC|purchase  |67347fb6-2e23-4695-8d32-03829cc6fdaf|
+-----------------------+----------+------------------------------------+



In [16]:
# Sort ASC topseller df by user_id and event 
sorted_by_user_event = topseller.orderBy(["user_id","event_type"], ascending= [True, True])

In [17]:
# Return result
sorted_by_user_event.show(5,truncate=False)

+-----------------------+----------------+--------+------------------------------------+
|event_time             |event_type      |user_id |user_session                        |
+-----------------------+----------------+--------+------------------------------------+
|2020-01-28 05:43:43 UTC|view            |4661182 |ce380fb8-bfdd-49a5-949f-eeac2c349af1|
|2020-01-28 05:41:23 UTC|view            |4661182 |ce380fb8-bfdd-49a5-949f-eeac2c349af1|
|2020-01-14 09:33:54 UTC|cart            |12961898|6cb94e2c-4db2-46ac-8c11-1ced638112bc|
|2020-01-13 11:36:50 UTC|cart            |12961898|91ceb8ab-fa0f-4929-b02e-9f36a19e73a5|
|2020-01-27 09:37:44 UTC|remove_from_cart|12961898|ea56a517-171d-4740-a9f7-f628f40274c8|
+-----------------------+----------------+--------+------------------------------------+
only showing top 5 rows



In [18]:
# Group df by user_id and event_type
event_type_count_user = sorted_by_user_event.groupby('user_id','event_type').count()

In [19]:
# New df with organised data counting event_type by user_id
sorted_event_type_count_user = event_type_count_user.orderBy(["user_id","event_type"], ascending= [True, True])

In [20]:
# Display new df
sorted_event_type_count_user.show(10)

+--------+----------------+-----+
| user_id|      event_type|count|
+--------+----------------+-----+
| 4661182|            view|    2|
|12961898|            cart|    2|
|12961898|remove_from_cart|    1|
|27692594|            cart|    1|
|31272873|            cart|    1|
|31272873|remove_from_cart|    1|
|46383252|            cart|    1|
|46383252|        purchase|    1|
|46383252|            view|    2|
|47076222|            view|    5|
+--------+----------------+-----+
only showing top 10 rows



In [25]:
# Convert PySpark Dataframe to Pandas
pandas_df = sorted_event_type_count_user.toPandas()
pandas_df.head()

,user_id,event_type,count
0,4661182,view,2
1,12961898,cart,2
2,12961898,remove_from_cart,1
3,27692594,cart,1
4,31272873,cart,1
